In [ ]:
import os
import glob
import json
import numpy as np
import cv2
from skimage.io import imread
from datetime import datetime
import pycocotools.mask as maskUtils  # Required for RLE encoding


In [ ]:
os.chdir("insert_path")

In [ ]:
# Define category ID for SAM2 format
CATEGORY_ID = 1  # Assuming single category for masks

MASK_EXT = "png"
ORIGINAL_EXT = "png"  # Change if needed
image_id = 0

def process_mask(mask_image_path, output_dir):
    """Convert a PNG mask into an individual JSON annotation file compatible with SAM2 using RLE encoding."""
    
    # Extract filename without extension
    base_name = os.path.basename(mask_image_path).split(".")[0]
    original_file_name = f"{base_name}.{ORIGINAL_EXT}"  # Use PNG extension
    
    # Read and binarize the mask
    mask_array = imread(mask_image_path)
    mask_binary = np.where(mask_array > 0, 1, 0).astype(np.uint8)  # Convert mask to binary (1s and 0s)
    
    # Get image dimensions
    height, width = mask_binary.shape
    
    # Encode mask using RLE (Run-Length Encoding)
    rle_mask = maskUtils.encode(np.asfortranarray(mask_binary))
    rle_mask["counts"] = rle_mask["counts"].decode("utf-8")  # Convert to string for JSON compatibility
    
    # Create image metadata
    image_info = {
        "image_id": int("".join(filter(str.isdigit, base_name))),  # Extract numeric ID from filename
        "license": 1,
        "file_name": original_file_name,
        "height": height,
        "width": width,
        "date_captured": datetime.now().isoformat(),
        "extra": {"name": original_file_name}
    }
    
    # Prepare annotation data
    annotation = {
        "id": 0,  # Using a single object per file
        "bbox": [0, 0, width, height],  # Whole image treated as bounding box
        "area": int(mask_binary.sum()),  # Area is the sum of pixels in mask
        "segmentation": {"counts": rle_mask["counts"], "size": [height, width]},  # RLE encoding
    }

    # Create individual JSON format
    json_output = {
        "image": image_info,
        "annotations": [annotation]
    }
    
    # Save JSON file
    json_filename = f"{base_name}.json"
    json_path = os.path.join(output_dir, json_filename)
    
    with open(json_path, "w") as json_file:
        json.dump(json_output, json_file, indent=4)

    print(f"Saved: {json_path}")

def convert_masks_to_json(mask_dir, output_json_dir):
    """Process all PNG masks and save individual JSON files."""
    os.makedirs(output_json_dir, exist_ok=True)
    
    for mask_image in glob.glob(os.path.join(mask_dir, f"*.{MASK_EXT}")):
        process_mask(mask_image, output_json_dir)

if __name__ == "__main__":
    mask_directory = "./test/masks"
    output_json_directory = "./test/json_masks"
    
    convert_masks_to_json(mask_directory, output_json_directory)
